In [25]:
import pandas as pd

In [37]:
ts_df = pd.read_csv('../ddi/data/complete_severity_reclassification.csv')

In [43]:
ts_df = ts_df[(ts_df['sub_system_severity']!='zzz_delete_0') & (ts_df['sub_system_severity']!='Delete -Moderate')]

In [44]:
ts_df['sub_system'] = ts_df['sub_system_severity'].apply(lambda x: x[:x.find('-') - 1])

In [79]:
ts_df['sub_system_severity'].unique()

array(['Bile duct,GallBladder And Pancreas -Moderate',
       'Bile duct,GallBladder And Pancreas -Severe',
       'Bladder Urethra -Moderate', 'Bladder Urethra -Severe',
       'Blood -Moderate', 'Blood -Severe', 'Blood Cells -Moderate',
       'Blood Cells -Severe', 'Blood Electrolytes Vitamins -Mild',
       'Blood Electrolytes Vitamins -Moderate', 'Blood Vessels -Mild',
       'Blood Vessels -Moderate', 'Blood Vessels -Severe',
       'Bone -Moderate', 'Bone -Severe', 'Brain Spinal Cord -Mild',
       'Brain Spinal Cord -Moderate', 'Brain Spinal Cord -Severe',
       'Breast -Mild', 'Breast -Moderate', 'Breast -Severe',
       'Bronchi Respiratory General -Mild',
       'Bronchi Respiratory General -Moderate',
       'Bronchi Respiratory General -Severe', 'Congenital -Moderate',
       'Congenital -Severe', 'Ears -Mild', 'Ears -Moderate', 'Eyes -Mild',
       'Eyes -Moderate', 'Gastrointestinal Tract General -Mild',
       'Gastrointestinal Tract General -Moderate', 'Hair OR Nails 

In [46]:
ts_df['sub_system'].unique()

array(['Bile duct,GallBladder And Pancreas', 'Bladder Urethra', 'Blood',
       'Blood Cells', 'Blood Electrolytes Vitamins', 'Blood Vessels',
       'Bone', 'Brain Spinal Cord', 'Breast',
       'Bronchi Respiratory General', 'Congenital', 'Ears', 'Eyes',
       'Gastrointestinal Tract General', 'Hair OR Nails', 'Hair', 'Heart',
       'Joints Connective Tissues', 'Kidneys', 'Liver', 'Lungs',
       'Motor Movement', 'Multiple Systemic General', 'Muscles Tendon',
       'Neurological', 'Nose', 'Oesophagus Stomach',
       'Oral Cavity Salivary Gland Throat', 'Psychiatric', 'Rectum Anus',
       'Female Reproductive System', 'General Reproductive System',
       'Male Reproductive System', 'Skin Subcutaneous Fat',
       'Small/Large Intestine',
       'Spleen, Bone Marrow, Lymph Node Tonsils Or Immune',
       'Thyroid Pituitary Gland Adrenal Parathyroids',
       'General Urinary System General'], dtype=object)

In [55]:
df = pd.DataFrame(columns=['Mild', 'Moderate', 'Severe'], index=ts_df['sub_system'].unique(), data=0)

In [49]:
import joblib

In [51]:
pipeline = joblib.load('../model.joblib')

In [57]:
df

,Mild,Moderate,Severe
"Bile duct,GallBladder And Pancreas",0,0,0
Bladder Urethra,0,0,0
Blood,0,0,0
Blood Cells,0,0,0
Blood Electrolytes Vitamins,0,0,0
Blood Vessels,0,0,0
Bone,0,0,0
Brain Spinal Cord,0,0,0
Breast,0,0,0
Bronchi Respiratory General,0,0,0


In [77]:
side_effect = ['Skin Subcutaneous Fat -Mild', 'Oral Cavity Salivary Gland Throat -Moderate']

In [75]:
for col in df:
    for row in df.index:
        if side_effect.count(row + ' -' + col) != 0:
            df.loc[row, col] = 10

In [76]:
df

,Mild,Moderate,Severe
"Bile duct,GallBladder And Pancreas",0,0,0
Bladder Urethra,0,0,0
Blood,0,0,0
Blood Cells,0,0,0
Blood Electrolytes Vitamins,0,0,0
Blood Vessels,0,0,0
Bone,0,0,0
Brain Spinal Cord,0,0,0
Breast,0,0,0
Bronchi Respiratory General,0,0,0


In [81]:
import plotly.graph_objects as go

In [86]:
fig = go.Figure(data=[go.Table(
    header=dict(values=['Location of side effects'] + list(df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df.index, df.Mild, df.Moderate, df.Severe],
               fill_color='lavender',
               align='center',
                  ))
])

fig.show()